In [1]:
# import the necessary packages
import numpy as np
import argparse
import imutils
import cv2
import scipy.special
def sort_contours(cnts, method="left-to-right"):
    # initialize the reverse flag and sort index
    reverse = False
    i = 0
    # handle if we need to sort in reverse
    if method == "right-to-left" or method == "bottom-to-top":
        reverse = True
    # handle if we are sorting against the y-coordinate rather than
    # the x-coordinate of the bounding box
    if method == "top-to-bottom" or method == "bottom-to-top":
        i = 1
    # construct the list of bounding boxes and sort them from top to
    # bottom
    boundingBoxes = [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingBoxes) = zip(*sorted(zip(cnts, boundingBoxes),
        key=lambda b:b[1][i], reverse=reverse))
    # return the list of sorted contours and bounding boxes
    return (cnts, boundingBoxes)

In [2]:
import cv2
import numpy as np
import numpy
import cv2 as cv
import pickle
score=cv2.imread('C:/Users/32665/Desktop/test2.jpg',0)
#score=cv2.imread('C:/Users/32665/Desktop/test2.jpg',0)
image_clone=score.copy()
gray=cv2.cvtColor(image_clone,cv2.COLOR_BAYER_BG2GRAY)
#image=color_seperate(image_clone)
thresh, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV)
binary=cv2.dilate(binary,np.ones((2,2),np.uint8),iterations =1)
rows,cols=binary.shape
scale = 40
#horizontal line recognize
kernel  = cv2.getStructuringElement(cv2.MORPH_RECT,(cols//scale,1))
eroded = cv2.erode(binary,kernel,iterations = 1)
#cv2.imshow("Eroded Image",eroded)
dilatedcol = cv2.dilate(eroded,kernel,iterations = 1)

 
#vertical line recognize
scale = 20
kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(1,rows//scale))
eroded = cv2.erode(binary,kernel,iterations = 1)
dilatedrow = cv2.dilate(eroded,kernel,iterations = 1)

 
#intersection 
bitwiseAnd = cv2.bitwise_and(dilatedcol,dilatedrow)


bitwiseAnd_clone=bitwiseAnd.copy()
score_clone=score.copy()
#cv2.imshow('',score_clone)
#cv2.waitKey(0)

In [3]:
# Using HSV to retrieve red handwritting number
def color_seperate(image):
    img_hsv = cv.cvtColor(image,cv.COLOR_BGR2HSV)
    rows, cols, channels = image.shape

    lower_red = np.array([0, 43, 46])
    upper_red = np.array([10, 255, 255])
    mask0 = cv.inRange(img_hsv,lower_red,upper_red)

    lower_red = np.array([156, 43, 46])
    upper_red = np.array([180, 255, 255])
    mask1 = cv.inRange(img_hsv,lower_red,upper_red)
    # connect two area
    mask = mask0 + mask1

    return(mask)

In [4]:
def findCentroid(ROI):
    M=cv2.moments(ROI)
    cX = int(M["m10"] / M["m00"])
    cY = int(M["m01"] / M["m00"])
    return(cX,cY)

In [5]:
# transfer the picture to MNIST formate
def Transe_MNist(ROI,h,w):
    img=ROI 
    ROI_cX,ROI_cY=findCentroid(img)
    if (h>3*w)&(ROI_cY<0.6*h):
        w_padding=(h-w)//3
        top_size,bottom_size,left_size,right_size=(0,0,w_padding,w_padding)
        wrap_ROI=cv2.copyMakeBorder(img, top_size, bottom_size, left_size, right_size, cv2.BORDER_CONSTANT, value=0)
    else:
        wrap_ROI=img

    ROI_20= cv2.resize(wrap_ROI,(20,20))
    cX,cY=findCentroid(ROI_20)
    affine_arr = np.float32([[1,0,14-cX],[0,1,14-cY]])
    res = cv2.warpAffine(ROI_20,affine_arr,(28,28))
    return(res)



In [6]:
class neuralNetwork :
    def __init__(self, inputNodes, hiddenNodes, outputNodes, learningRate,Wih_finished,Who_finished,b1_finished,b2_finished):
# set number of nodes in each input, hidden,output layer
         self.iNodes=inputNodes
         self.hNodes=hiddenNodes
         self.oNodes=outputNodes
 #learning rate
         self.lr=learningRate
#set up weight matrix which meet the require of Gaussian distribution
#function(argument1,argument2,argument3)
#argument1 refers to means of Gaussian distribution 
#argument2 refers to standar deviation
#argument3 refers to dim of matrix 
#weight input to hidden layer
         self.Wih=Wih_finished
#weight hidden to output layer
         self.Who=Who_finished
         self.b1=b1_finished.reshape(500,1)
         self.b2=b2_finished.reshape(10,1)
#Activation function sigmoid()
         self.activation_function= lambda x: scipy.special.expit(x)     
#Block of query() 
#refers to this block accepts the input of NN and gives feedback 
    def query(self,input_list):
    #ndmin refers to min dim=2
    #T refers to transpose
            inputs=numpy.array(input_list,ndmin=2).T
    # Wih·I=X1 
            hidden_inputs=numpy.dot(self.Wih,inputs)+self.b1
    #hidden_output=sigmoid(X1)=sigmoid(Wih·I)
            hidden_outputs=self.activation_function(hidden_inputs)
    #Who·O=X2
            final_input=numpy.dot(self.Who,hidden_outputs)+self.b2
    #final_output=sigmoid(X2)=sigmoid(Who·X2)
            final_outputs=self.activation_function(final_input)
            return (hidden_outputs)

In [7]:
Who_finished = numpy.loadtxt('C:/Users/32665/Desktop/train_model/97.7sigrotate+b500hid/Who_new_sigb_rotate_97.7%.csv',delimiter=',')
Wih_finished=numpy.loadtxt('C:/Users/32665/Desktop/train_model/97.7sigrotate+b500hid/Wih_new_sigb_rotate_97.7%.csv',delimiter=',')
b1_finished=numpy.loadtxt('C:/Users/32665/Desktop/train_model/97.7sigrotate+b500hid/b1_new_sigb_rotate_97.7%.csv',delimiter=',')
b2_finished=numpy.loadtxt('C:/Users/32665/Desktop/train_model/97.7sigrotate+b500hid/b2_new_sigb_rotate_97.7%.csv',delimiter=',')

In [8]:
# load model
file = open("C:/Users/32665/Desktop/transform_training/model/transform_learning_svm.pickle", "rb")
predictor=pickle.load(file)
file.close()

In [9]:
input_nodes=784
hidden_nodes=500
output_nodes=10
# set learning rate 
learning_rate = 0.1
n=neuralNetwork(input_nodes,hidden_nodes,output_nodes,learning_rate,Wih_finished,Who_finished,b1_finished,b2_finished)

In [10]:
score_color=cv2.imread('C:/Users/32665/Desktop/test2.jpg',1)
#score_color=cv2.imread('C:/Users/32665/Desktop/test2.jpg',1)
score_color_clone=score_color.copy()

In [11]:
# filter the noise
rect_x=[]
contours_rec, hierarchy_rec = cv2.findContours(dilatedrow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
for c in contours_rec:
    x, y, w, h = cv2.boundingRect(c)
    border = [(x,y+h), (x,y),(x+w,y),(x+w,y+h)]
    border=numpy.int0(border)
    if (h>100)&(w<30):
        #print(h)
        rect_x.append(x)
    

In [12]:
y=0
h=130
w=80
ROI_rects=[]
for i in rect_x:
    x=i
    ROI_rect=score_color_clone[y:y+h,x:x+w]
    ROI_rects.append(ROI_rect)


In [13]:
for ROI_rect in ROI_rects:
    image=ROI_rect   
    image_clone=image.copy()
    ROI=color_seperate(image_clone)
    contours_ROI, hierarchy = cv2.findContours(ROI, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if len(contours_ROI) > 0:
        contours, boundingBoxes = sort_contours(contours_ROI, method="top-to-bottom")

In [14]:
#Calculate the score 

In [15]:
score=0
for ROI_rect in ROI_rects:
    score_list=[]
    image=ROI_rect   
    image_clone=image.copy()
    ROI_unerode=color_seperate(image_clone)
    ROI=cv2.erode(ROI_unerode,np.ones((2,2),np.uint8),iterations=1)
    contours_ROI, hierarchy = cv2.findContours(ROI, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if len(contours_ROI) > 0:
        contours_ROI,_=sort_contours(contours_ROI,"left-to-right")
    # find boundary
    MaxArea=ThreshArea=320
    MinArea=2000
    borders=[]
    for c in contours_ROI:
            # ger the region of interest
        x, y, w, h = cv2.boundingRect(c)
        if w*h > ThreshArea:
            border = [(x, y+h), (x,y),(x+w,y),(x+w,y+h)]
            border=numpy.int0(border)
            cv2.drawContours(image_clone,[border],-1, (0, 0, 255), 1)
            ROI_new=ROI[y:y+h,x:x+w]
            inputs=Transe_MNist(ROI_new,h,w)
            test_image=inputs.reshape(1,-1)/255*0.99+0.01
            inputs_500=n.query(test_image)
            number=int(predictor.predict(inputs_500.reshape(1,-1)))
            score_list.append(number)
            print(number)
            result=cv2.putText(image_clone,str(number),(x,y+h), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 1)
    if len(score_list)==2:
        unit_score=score_list[0]*10+score_list[1]
    elif len(score_list)==1:
        unit_score=score_list[0]
    else:
        unit_score=0
    score+=unit_score
print("Full score is："+str(score))
cv2.imshow("full",image_clone)

cv2.waitKey(0)


7
3
2
3
1
0
1
2
总分为：64


-1